Composite frames of image to show the trajectory of each task 

In [11]:
import numpy as np
from PIL import Image, ImageChops, ImageEnhance

In [26]:
# select important frame from the image
img_1 = Image.open('./task_3/image_0.png')
img_2 = Image.open('./task_3/image_22.png')
seg_1 = np.load('./task_3/seg_index_0.npy')
seg_2 = np.load('./task_3/seg_index_22.npy')

images = [img_1, img_2]
masks=[seg_1, seg_2]

print(type(img_1))
print(type(seg_1))

<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>


In [27]:
# robot arm value is 1 in the segmentation image
# background value is 0 in the segmentation image
# we can use the segmentation image to extract the robot arm
def combine_images_with_composite_for_arm(images, masks, arm_index=1, box_index=3, alpha_start=0.5, alpha_end=1.0):
    """
    Combines multiple images, selectively using the segments corresponding to the robotic arm.

    Args:
    - images (list of Image): List of PIL Image objects to combine.
    - masks (list of np.array): List of grayscale masks corresponding to the images, where the robotic arm has index `arm_index`.
    - arm_index (int): The index value in the mask representing the robotic arm.
    - alpha_start (float): Starting alpha value for the first image. Must be between 0.0 and 1.0.
    - alpha_end (float): Ending alpha value for the last image. Must be between 0.0 and 1.0.

    Returns:
    - Image: The combined image with only the robotic arm segments.
    """
    if len(images) == 0 or len(masks) == 0:
        raise ValueError("The list of images or masks is empty.")
    if len(images) != len(masks):
        raise ValueError("The number of images and masks must be the same.")
    
    # Ensure alpha values are valid
    if not (0.0 <= alpha_start <= 1.0) or not (0.0 <= alpha_end <= 1.0):
        raise ValueError("Alpha values must be between 0.0 and 1.0.")

    # Calculate the alpha increment step
    alpha_step = (alpha_end - alpha_start) / (len(images) - 1)

    # Start with a transparent base image
    base_image = images[0].convert("RGBA")

    # Iterate over the images with increasing alpha
    current_alpha = alpha_start
    for img, mask in zip(images[1:], masks[1:]):
        img = img.convert("RGBA")        

        # Create a mask where the robot arm's index is 1 and block mask is 3
        arm_mask = (mask == arm_index).astype(np.uint8) * 255
        box_mask = (mask == box_index).astype(np.uint8) * 255

        mask = Image.fromarray(arm_mask+box_mask, "L")

        # Apply the transparency to the mask
        alpha_mask = ImageEnhance.Brightness(mask).enhance(current_alpha)

        # Composite the current image (only arm) onto the base image using the mask
        base_image = Image.composite(img, base_image, alpha_mask)
        
        # Increment the alpha value for the next image
        current_alpha += alpha_step

    return base_image


In [28]:
combined_image = combine_images_with_composite_for_arm(images, masks, arm_index=1, alpha_start=1, alpha_end=0.4)
combined_image.save("grasping_task.png")
combined_image.show()

** Message: 11:20:08.623: main.vala:507: Starting session with system profile
